# Fixmydata tutorial: cleaning built-in datasets

This notebook demonstrates how to explore and clean the bundled sample datasets using the `Fixmydata` utilities. Each section mirrors a typical data quality workflow so you can adapt the snippets to your own projects.

## Prerequisites
- Install dependencies from `requirements.txt`.
- Ensure the project root is on your Python path so `Fixmydata` can be imported directly.

In [1]:
import sys
import pandas as pd
from pathlib import Path

# Ensure project root is on the Python path
ROOT = Path().resolve().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

DATA_DIR = ROOT / 'datasets'

from Fixmydata import DataCleaner, DataValidator, OutlierDetector

## 1. Load the Titanic-style passenger data

We will use `datasets/tested.csv`, which mirrors the familiar Titanic competition data.

In [2]:
titanic_path = DATA_DIR / 'tested.csv'
titanic_df = pd.read_csv(titanic_path)

print(titanic_df.shape)
titanic_df.head()

(418, 12)


   PassengerId  Survived  Pclass                                          Name     Sex  ...  Parch   Ticket     Fare Cabin  Embarked
0          892         0       3                              Kelly, Mr. James    male  ...      0   330911   7.8292   NaN         Q
1          893         1       3              Wilkes, Mrs. James (Ellen Needs)  female  ...      0   363272   7.0000   NaN         S
2          894         0       2                     Myles, Mr. Thomas Francis    male  ...      0   240276   9.6875   NaN         Q
3          895         0       3                              Wirz, Mr. Albert    male  ...      0   315154   8.6625   NaN         S
4          896         1       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  ...      1  3101298  12.2875   NaN         S

[5 rows x 12 columns]

### Inspect missing values
Before cleaning, it is useful to see which columns contain gaps.

In [3]:
titanic_df.isnull().sum().to_frame('missing_values')

             missing_values
PassengerId               0
Survived                  0
Pclass                    0
Name                      0
Sex                       0
Age                      86
SibSp                     0
Parch                     0
Ticket                    0
Fare                      1
Cabin                   327
Embarked                  0

## 2. Clean the passenger data
We will standardize column names, fill missing numeric values with the median, fill categorical gaps with the mode, and remove duplicates. The `DataCleaner` instance keeps track of the working DataFrame internally.

In [4]:
cleaner = DataCleaner(titanic_df)

# Normalize headers for easier downstream processing
cleaner.standardize_columns()

# Replace missing numeric values with medians
cleaner.fill_missing(strategy='median', columns=['age', 'fare'])

# Fill categorical gaps using the most frequent values
cleaner.fill_missing(strategy='mode')

# Drop accidental duplicate rows if any
titanic_clean = cleaner.remove_duplicates()

titanic_clean.head()

   passengerid  survived  pclass                                          name  ...   ticket     fare            cabin  embarked
0          892         0       3                              Kelly, Mr. James  ...   330911   7.8292  B57 B59 B63 B66         Q
1          893         1       3              Wilkes, Mrs. James (Ellen Needs)  ...   363272   7.0000  B57 B59 B63 B66         S
2          894         0       2                     Myles, Mr. Thomas Francis  ...   240276   9.6875  B57 B59 B63 B66         Q
3          895         0       3                              Wirz, Mr. Albert  ...   315154   8.6625  B57 B59 B63 B66         S
4          896         1       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  ...  3101298  12.2875  B57 B59 B63 B66         S

[5 rows x 12 columns]

### Validate the cleaned data
`DataValidator` can assert common expectations. Here we ensure the DataFrame is non-empty and that passenger ages fall inside a reasonable range.

In [5]:
validator = DataValidator(titanic_clean)
validator.validate_non_empty()
validator.validate_range('age', 0, 90)

titanic_clean[['age', 'fare']].describe()

              age        fare
count  418.000000  418.000000
mean    29.599282   35.576535
std     12.703770   55.850103
min      0.170000    0.000000
25%     23.000000    7.895800
50%     27.000000   14.454200
75%     35.750000   31.471875
max     76.000000  512.329200

### Detect and remove outliers
We can use `OutlierDetector` to filter extreme values. The IQR method is robust for skewed distributions like fares.

In [6]:
detector = OutlierDetector(titanic_clean)
titanic_iqr = detector.iqr_outliers()

print('Original rows:', len(titanic_clean))
print('Rows after IQR filtering:', len(titanic_iqr))

titanic_iqr[['age', 'fare']].describe()

Original rows: 418
Rows after IQR filtering: 281


              age        fare
count  281.000000  281.000000
mean    28.272242   15.618090
std      7.876031   12.818909
min     12.000000    0.000000
25%     24.000000    7.775000
50%     27.000000    8.662500
75%     30.000000   21.000000
max     54.000000   65.000000

## 3. Explore the USA housing data
The `USA Housing Dataset.csv` contains home sale information. The same cleaners can be applied to prepare the data for modeling.

In [7]:
housing_path = DATA_DIR / 'USA Housing Dataset.csv'
housing_df = pd.read_csv(housing_path)
housing_df.head()

                  date      price  bedrooms  bathrooms  ...                   street       city  statezip  country
0  2014-05-09 00:00:00   376000.0       3.0       2.00  ...  9245-9249 Fremont Ave N    Seattle  WA 98103      USA
1  2014-05-09 00:00:00   800000.0       4.0       3.25  ...         33001 NE 24th St  Carnation  WA 98014      USA
2  2014-05-09 00:00:00  2238888.0       5.0       6.50  ...         7070 270th Pl SE   Issaquah  WA 98029      USA
3  2014-05-09 00:00:00   324000.0       3.0       2.25  ...           820 NW 95th St    Seattle  WA 98117      USA
4  2014-05-10 00:00:00   549900.0       5.0       2.75  ...        10834 31st Ave SW    Seattle  WA 98146      USA

[5 rows x 18 columns]

### Clean housing records and compute quick insights
We standardize column names, fill any numeric gaps with column means, and check the relationship between living area and sale price after removing Z-score outliers.

In [8]:
housing_cleaner = DataCleaner(housing_df)
housing_cleaner.standardize_columns()
housing_cleaner.fill_missing(strategy='mean')
housing_base = housing_cleaner.remove_duplicates()

housing_detector = OutlierDetector(housing_base)
housing_no_outliers = housing_detector.z_score_outliers(threshold=3)

price_sqft_corr = housing_no_outliers['price'].corr(housing_no_outliers['sqft_living'])
print(f'Correlation between price and square footage: {price_sqft_corr:.3f}')
housing_no_outliers[['price', 'sqft_living', 'bedrooms', 'bathrooms']].describe()

Correlation between price and square footage: 0.611


              price  sqft_living     bedrooms   bathrooms
count  3.805000e+03  3805.000000  3805.000000  3805.00000
mean   4.991892e+05  2019.579763     3.349803     2.09159
std    2.716459e+05   786.018440     0.855091     0.70747
min    0.000000e+00   370.000000     1.000000     0.75000
25%    3.128910e+05  1430.000000     3.000000     1.75000
50%    4.448450e+05  1910.000000     3.000000     2.25000
75%    6.200000e+05  2500.000000     4.000000     2.50000
max    2.300000e+06  4960.000000     6.000000     4.50000

## Next steps
- Swap in your own CSV paths and reuse the same cleaning steps.
- Try different fill strategies (mean/median/mode) depending on the data type.
- Adjust outlier thresholds to balance robustness and recall.
- Add additional validation checks before training models or generating reports.